In [14]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off


In [15]:
import os
import textwrap
from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install groq


In [18]:
os.environ["GROQ_API_KEY"] = ""

# def print_response(response):

def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))



# Document parsing

In [25]:
#Document parsing (cleaning the document)
instruction = """The provided document is the Constitution of Nepal 2072. This document encompasses all the legal frameworks, guidelines, and principles governing the country. It defines the structure of the state, the distribution of powers between different levels of government, and the fundamental rights and duties of citizens.

When answering questions based on this document, please follow these guidelines:
- Be precise and concise in your responses.
- Ensure that the information is accurate and directly relevant to the question.
- Highlight key articles, sections, or provisions where applicable.
- Provide context to your answers when necessary, explaining the implications or importance of specific provisions.
- Maintain a neutral and informative tone, avoiding any personal opinions or interpretations.

The goal is to provide clear and informative answers that help the user understand the specific aspects of the Constitution of Nepal 2072."""

parser = LlamaParse(
    api_key = '',
    result_type = 'markdown',
    parsing_instruction = instruction,
    max_timeout = 5000,

)

llama_parse_document = await parser.aload_data('/content/drive/My Drive/Constitution_Nepal.pdf')

Started parsing the file under job_id cac11eca-e9a7-4737-a062-cc2fd6bbae64


In [30]:
parsed_doc = llama_parse_document[0]
Markdown(parsed_doc.text[:1000])

# The Constitution of Nepal

|Date of Publication in Nepal Gazette|20 September 2015 (2072.6.3)|
|---|---|
|The Constitution of Nepal (First Amendment), 2016 (2072)|28 February 2016 (2072.11.16)|

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingua

In [31]:
from pathlib import Path

In [32]:
# save the file to
document_path = Path('/content/drive/MyDrive/parsed_document.md')
with document_path.open("a") as f:
  f.write(parsed_doc.text)

# Vector embedding

In [35]:
# Handle markdown file
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [36]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
print(docs[0].page_content)

The Constitution of Nepal

Date of Publication in Nepal Gazette 20 September 2015 (2072.6.3) The Constitution of Nepal (First Amendment), 2016 (2072) 28 February 2016 (2072.11.16)

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingual, multi-religio

In [38]:
# model embedding
embeddings = FastEmbedEmbeddings(
    model_name = "BAAI/bge-base-en-v1.5")
qdrant = Qdrant.from_documents(docs,
                               embeddings,
                               path = '/content/drive/MyDrive/qdrant_db',
                               collection_name = "document_embeddings")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [39]:
%%time
query = "What is the constitution?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 335 ms, sys: 6.95 ms, total: 342 ms
Wall time: 388 ms


In [40]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: The Constitution of Nepal

Date of Publication in Nepal Gazette 20 September 2015 (2072.6.3) The Constitution of Nepal (First Amendment), 2016 (2072) 28 February 2016 (2072.11.16)

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's so

score: 0.6272225604825328
--------------------------------------------------------------------------------

text: (2) Each House of the Federal Parliament shall, subject to this Constitution, have full power to regulate and decide its internal business, and the concerned House shall have the exclusive right to decide whether or not any of its proceedings is regular or

score: 0.6208846091448812
--------------------------------------------------------------------------------

text: Provided that he or she may exercise his or her casting vote in the case of a tie.

Privileges of State Assembly:

(1) There shall be full freedom of speech in the State Assembly, subject to this Constitution, and no member shall be arrested, detaine